# __Search Text by Text__

- Tutorial Difficulty : ★☆☆☆☆
- 7 min read 
- Languages : [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location : tutorial_en/thanosql_search/search_text_by_text.ipynb   
- References : [Worrd Embeddings: LEXICAL SEMANTICS Encoding](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Text Vectorization</h4>
    <p>Computers cannot directly interpret human language (natural language). Therefore, a process for converting natural language into numerical data that can be recognized by computers is required. In the field of natural language processing, embedding refers to the result of converting human natural language into a vectorized format, which is a form that can be understood by machines.</p>
</div>

Techniques for converting natural language into embeddings are largely divided into statistical techniques and artificial neural network-based techniques. ThanoSQL provides a method to train a text vectorization model using self-supervised learning.

<div class="admonition note">
    <h4 class="admonition-title">In this tutorial</h4>
    <p>👉 Uses <mark style="background-color:#FFD79C">movie review data</mark>. The data consists of movie review texts and label values. However, because we are demonstrating self-supervised learning, this tutorial does not use label values. By training a model with 4,000 movie reviews, we will be able to search text by text and extract the importance of each word from a given movie review. </p>
</div>

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [3]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite" : Determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

In [4]:
%%thanosql
COPY movie_review_train
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_train.csv"

Success


In [5]:
%%thanosql
COPY movie_review_test 
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"overwrite" : Determines whether to overwrite a table if it already exists. If set as True, the old table is replaced with the new table (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1.Check Dataset__

To create a movie review text search model, we use the <mark style="background-color:#FFEC92 ">movie_review_train</mark> table from the ThanoSQL database. To check the table's contents, run the following query.

In [16]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5 

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data</h4>
   <ul>
      <li><mark style="background-color:#D7D0FF ">review</mark>: movie review in text format</li>
      <li><mark style="background-color:#D7D0FF ">sentiment</mark> : target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __3. Create a Text Vectorization Model__

To create a text search model with the name <mark style="background-color:#E9D7FD ">movie_text_search_model</mark> using the <mark style="background-color:#FFEC92 ">movie_review_train</mark> dataset, run the following query.  
(Estimated time required for query execution: 2 min)

In [7]:
%%thanosql
BUILD MODEL movie_text_search_model
USING SBERTEn
OPTIONS (
    text_col="review",
    overwrite=True
)
AS
SELECT *
FROM movie_review_train

Building model...
Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <mark style="background-color:#E9D7FD">movie_text_search_model</mark>.</li>
        <li>"<strong>USING</strong>" specifies <mark style="background-color:#E9D7FD">SBERTEn</mark> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"text_col" : a column containing movie review data in the data table.</li>
            <li>"epochs" : number of times to train with the training dataset (int, DEFAULT : 1)</li>
            <li>"batch_size" : the size of dataset bundle utilized in a single cycle of training. (int, DEFAULT : 16)</li> 
            <li>"learning_rate" : the learning rate of the model (float, DEFAULT : 3e-5)</li> 
            <li>"train" : determines whether to use a pretrained model. If specified as False, the pretrained model is used. (True|False, DEFAULT : True)</li> 
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (True|False, DEFAULT : False)</li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the `movie_review_test` texts run the following "__CONVERT USING__" query. The vectorized results are stored in a column named <mark style="background-color:#D7D0FF">movie_text_search_model_sberten</mark> in the `movie_review_test` table.

In [8]:
%%thanosql
CONVERT USING movie_text_search_model
OPTIONS (
    text_col="review",
    table_name="movie_review_test",
    batch_size=32
    )
AS 
SELECT *
FROM movie_review_test

,review,sentiment,movie_text_search_model_sberten
0,"I read the book before seeing the movie, and t...",positive,"[-0.0035803898, -0.008503916700000001, 0.02663..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[9.95681e-05, -0.0446751118, 0.0422799364, -0...."
2,"Yesterday I attended the world premiere of ""De...",positive,"[0.0024927575, -0.0627047122, -0.0220359992, 0..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[-0.0074431906, -0.0184612498, 0.0346845426, 0..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[-0.0121047432, -0.08736537400000001, -0.01382..."
...,...,...,...
995,I remember catching this movie on one of the S...,negative,"[-0.0081995381, -0.022497111900000002, 0.00271..."
996,CyberTracker is set in Los Angeles sometime in...,negative,"[-0.0258360021, -0.0056749573, 0.0006066648, 0..."
997,"There is so much that is wrong with this film,...",negative,"[-0.0285293236, 0.0241474006, -0.0057142889, -..."
998,"I am a firm believer that a film, TV serial or...",positive,"[0.0242064726, -0.0032963406000000002, -0.0172..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <code>movie_text_search_model</code> as an algorithm for text vectorizaion.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text vectorizaion.
        <ul>
            <li>"text_col" : a column containing movie review data in the data table.</li>
            <li>"table_name" : the table name to be stored in the ThanoSQL database.</li>
            <li>"batch_size" : the size of dataset bundle utilized in a single cycle of training.</li> 
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Texts__

This step uses the <mark style="background-color:#E9D7FD">movie_text_search_model</mark> text vectorization model and test table to search for similar texts.

In [13]:
%%thanosql
SELECT review, sentiment, movie_text_search_model_sberten_similarity1 as score
FROM (
    SEARCH TEXT text="This movie was my favorite movie of all time"
    USING movie_text_search_model
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

Searching...


,review,sentiment,score
0,I have loved this movie since I saw it in the ...,positive,0.638331
1,"I always loved this film. The music,story and ...",positive,0.605845
2,"First time I saw this great movie and Alyssa, ...",positive,0.602958
3,I loved this movie. It is a definite inspirati...,positive,0.599915
4,This movie was the second movie I saw on the c...,positive,0.596825
5,This film lingered and lingered at a small mov...,positive,0.588338
6,This movie was very good because it remember w...,positive,0.586384
7,I saw this movie for the first time in 1988 wh...,positive,0.582169
8,This is a delightful film. Elizabeth Taylor do...,positive,0.578149
9,What can I say?? This movie has it all...Roman...,positive,0.576739


In [14]:
%%thanosql
SELECT review, sentiment, movie_text_search_model_sberten_similarity1 as score
FROM (
    SEARCH TEXT text="The movie was unsatisfactory"
    USING movie_text_search_model
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

Searching...


,review,sentiment,score
0,"While the dog was cute, the film was not. It w...",negative,0.707552
1,"A gave it a ""2"" instead of a ""1"" (awful) becau...",negative,0.704344
2,There was absolutely nothing in this film that...,negative,0.703998
3,Just plain terrible. Nick and Michael are WAY ...,negative,0.703163
4,"If this movie would have been in English, all ...",negative,0.702559
5,Badly made. Dreadful acting and an ending that...,negative,0.689564
6,"I knew this movie wasn't going to be amazing, ...",negative,0.685655
7,"Well, I remember when the studio sacked Schrad...",negative,0.675767
8,This movie was pointless. I can't even call it...,negative,0.672833
9,4 out of 10. This film was neither funny as a ...,negative,0.669799


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [images|audio|videos|texts|keywords]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for the searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __4. Extract Keywords from Texts__

This step uses the <mark style="background-color:#E9D7FD">movie_text_search_model</mark> text vectorization model and test table to extract keywords from the texts.

In [17]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM movie_review_test
LIMIT 10 OFFSET 40

Searching...


,review,sentiment,movie_text_search_model_sberten,keyword
0,"Like an earlier commentor, I saw it in 1980 an...",positive,"[-0.008661733000000001, -0.0488559045, 0.00187...","{'keyword': ['the gripping story', 'dramatic m..."
1,"I'm a fan of Matthew Modine, but this film--wh...",negative,"[-0.0112550212, 0.0041147466, -1.364e-07, -0.0...","{'keyword': ['watched it', 'screenwriter and d..."
2,Paul Lukas played a Russian intellectual makin...,positive,"[0.0207437705, -0.050762143, -0.01628353630000...","{'keyword': ['deadpan comic actor', 'name stan..."
3,"to be honest, i didn't watch all of the origin...",negative,"[0.0070642396, -0.0065534478, -0.0103744725, 0...","{'keyword': ['other vampire movie', 'bad it wa..."
4,Police Squad! (1982) was a funny show that end...,positive,"[-0.0234945826, -0.040255721700000004, 0.00883...","{'keyword': ['comedy hit airplane', 'police sq..."
5,I am still shuddering at the thought of EVER s...,negative,"[-0.0264584627, -0.0002144384, 0.004705308, -0...","{'keyword': ['better action movies', 'but the ..."
6,Gregory Peck gives a brilliant performance in ...,positive,"[0.0078235855, 0.0121560059, -0.0218212046, -0...","{'keyword': ['a brilliant performance', 'this ..."
7,I first flicked onto the LoG accidentally one ...,positive,"[-0.0272676237, -0.0017859461, 0.0039674025, 0...","{'keyword': ['watch it', 'the fast show', 'hum..."
8,I didn't know what to make of this film. I gue...,positive,"[-0.0384476818, -0.0128671648, 0.013419711, -0...","{'keyword': ['view the film', 'think to apprec..."
9,A family looking for some old roadside attract...,negative,"[0.0140419025, -0.036462456000000004, -0.01798...","{'keyword': ['poorly made horror', 'beloved fr..."


In [18]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
)

Searching...


,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,"[film very haunting, best adaptations out, tal...","[0.49860000000000004, 0.42610000000000003, 0.4..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[9 11 hosted, the twin towers, television show...","[0.5721, 0.5537000000000001, 0.389700000000000..."
2,"Yesterday I attended the world premiere of ""De...",positive,"[she meets jared, gritty rape scene, innocent ...","[0.4808, 0.38970000000000005, 0.2985, 0.205500..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[michael jackson film, music video of, liked t...","[0.5556, 0.42210000000000003, 0.3995, 0.274400..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[series the ghetto, brother jermaine brandon, ...","[0.39940000000000003, 0.33690000000000003, 0.3..."
5,Tipping the Velvet (2002) (TV) was directed by...,positive,"[protagonist nan astley, directed by geoffrey,...","[0.4555, 0.4006, 0.2677, 0.2137, 0.1925]"
6,The Stock Market Crash of 1929 and the Depress...,positive,"[james cagney in, favorite busby berkeley, dor...","[0.49870000000000003, 0.4056, 0.3929, 0.3773, ..."
7,I want to clarify a few things. I am not famil...,negative,"[an opportunistic movie, taiwan censorship and...","[0.5604, 0.3019, 0.26940000000000003, 0.2106, ..."
8,This is a nice movie with good performances by...,positive,"[spanish cinema, is very good, great directors...","[0.5735, 0.48810000000000003, 0.42260000000000..."
9,"Once a month, I invite a few friends over for ...",negative,"[retarded movie night, zombie i personally, jo...","[0.545, 0.32230000000000003, 0.3074, 0.2487, 0..."


In [20]:
%%thanosql 
SELECT * FROM (
    SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
        FROM (
            SEARCH KEYWORD 
            USING movie_text_search_model
            OPTIONS (
                text_col="review",
                use_stopwords=True
                )
            AS 
            SELECT * 
            FROM movie_review_test
            LIMIT 10
        )
    ) 
WHERE score > 0.5

Searching...


,review,sentiment,keywords,score
0,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,9 11 hosted,0.5721
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,the twin towers,0.5537
2,Moonwalker is a Fantasy Music film staring Mic...,positive,michael jackson film,0.5556
3,This is a nice movie with good performances by...,positive,spanish cinema,0.5735
4,I want to clarify a few things. I am not famil...,negative,an opportunistic movie,0.5604
5,"Once a month, I invite a few friends over for ...",negative,retarded movie night,0.5450


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD [images|audio|videos|texts|keywords]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for the text vectorizaion.
            <ul>
                <li>"lang" : en, ko</li>
                <li>"text_col" :  a column containing movie review data in the data table</li>
                <li>"ngram_range" : minimum and maximum number of words for each keyword. ex) [1, 3]. In most situations, keywords are extracted according to the maximum number of words. (list[int, int], DEFAULT : [1, 2])</li>
                <li>"top_n" : number of keywords to be extracted, in order of highest similarity (int, DEFAULT : 5)</li>
                <li>"diversity" : variety of keywords to be extracted. The higher the value, the more diverse the keywords will be. 0 <= diversity <= 1 (float, DEFAULT : 0.5)</li>
                <li>"use_stopwords" : whether to exclude words that do not have a significant meaning (True|False, DEFAULT : True)</li>
                <li>"threshold" : minimum value of similarity value of keywords to be extracted. (float, DEFAULT : 0.0)</li>
            </ul>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __5. Combine the Two Methods__

In [21]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, movie_text_search_model_sberten_similarity1 as score
    FROM (
        SEARCH TEXT text="The greatest movie of all time"
        USING movie_text_search_model
        AS 
        SELECT * 
        FROM movie_review_test
        )
    ORDER BY score DESC 
    LIMIT 10
)

Searching...
Searching...


,review,sentiment,score,keyword
0,For those out there that like to think of them...,positive,0.574758,"{'keyword': ['well paced movie', 'who love fil..."
1,This is the best Chinese movie I have ever see...,positive,0.569135,"{'keyword': ['best chinese movie', 'beautiful ..."
2,This is without a doubt the greatest film ever...,positive,0.568749,"{'keyword': ['greatest film ever', 'decipher t..."
3,A labor of love. Each frame is picture perfect...,positive,0.555554,"{'keyword': ['recommend this film', 'the remar..."
4,This is the kind of film for a snowy Sunday af...,positive,0.555060,"{'keyword': ['hours wonderful performances', '..."
5,This film lingered and lingered at a small mov...,positive,0.554078,"{'keyword': ['warming film', 'it a comedy', 'c..."
6,"This film immediately catches the eye, with th...",positive,0.549254,"{'keyword': ['film', 'compelling story', 'love..."
7,This is without a doubt the most poorly though...,negative,0.545353,"{'keyword': ['give this movie', 'the invention..."
8,The movie Angels of the Universe is a pure mas...,positive,0.541722,"{'keyword': ['best foreign film', 'movie angel..."
9,I was lured to see this movie by its starpower...,negative,0.534548,"{'keyword': ['modern thriller', 'the bad guys'..."


In [23]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, movie_text_search_model_sberten_similarity1 as score
        FROM (
            SEARCH TEXT text="Such a romatic movie"
            USING movie_text_search_model
            AS 
            SELECT * 
            FROM movie_review_test
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)

Searching...
Searching...


,review,sentiment,keywords,score
0,This is the kind of film for a snowy Sunday af...,positive,"[hours wonderful performances, a family film, ...","[0.5846, 0.5518000000000001, 0.414000000000000..."
1,Burlinson and Thornton give an outstanding per...,positive,"[the horse scenes, love this movie, are absolu...","[0.6133000000000001, 0.5341, 0.429600000000000..."
2,"""Crush"" examines female friendship, for the mo...",positive,"[crush examines female, film, emotional truth ...","[0.48450000000000004, 0.4355, 0.3473, 0.296, 0..."
3,i would have to say that this is the first qua...,positive,"[quality romantic comedy, movie was well, end ...","[0.5969, 0.49910000000000004, 0.3899, 0.179800..."
4,"""Casomai"" is a masterful tale depicting the st...",positive,"[best italian movies, masterful tale depicting...","[0.6425000000000001, 0.5043, 0.323200000000000..."
5,"This film immediately catches the eye, with th...",positive,"[film, compelling story, love and the, colours...","[0.5419, 0.4838, 0.3158, 0.28350000000000003, ..."
6,A labor of love. Each frame is picture perfect...,positive,"[recommend this film, the remarkable story, th...","[0.636, 0.5346000000000001, 0.3505000000000000..."
7,"A gentle story, hinting at fury, with a redemp...",positive,"[well executed cinematographers, film score, t...","[0.5819, 0.5658000000000001, 0.5315, 0.4884, 0..."
8,Forget the recent dire American remake which s...,positive,"[the intricate screenplay, deftly drawn romant...","[0.5155000000000001, 0.5103, 0.440600000000000..."
9,There are not many movies around that have giv...,positive,"[wonderful fantasy spectacle, bride stardust i...","[0.5861000000000001, 0.36150000000000004, 0.28..."


In [28]:
%%thanosql
SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
FROM (
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, movie_text_search_model_sberten_similarity1 as score
        FROM (
            SEARCH TEXT text="The best action movie"
            USING movie_text_search_model
            AS 
            SELECT * 
            FROM movie_review_test
            WHERE review LIKE '%%gun%%'
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)
WHERE score > 0.45

Searching...
Searching...


,review,sentiment,keywords,score
0,As a veteran screen writing instructor at Rich...,positive,decent films i,0.5256
1,As a veteran screen writing instructor at Rich...,positive,good old days,0.2753
2,As a veteran screen writing instructor at Rich...,positive,who will happily,0.2280
3,As a veteran screen writing instructor at Rich...,positive,33 in dallas,0.1801
4,As a veteran screen writing instructor at Rich...,positive,which betty white,0.1790
5,This is quite an unusual and unique little wes...,positive,western genre film,0.5798
6,This is quite an unusual and unique little wes...,positive,movie i liked,0.3852
7,This is quite an unusual and unique little wes...,positive,other surprising actors,0.3839
8,This is quite an unusual and unique little wes...,positive,involved quite amazing,0.3069
9,This is quite an unusual and unique little wes...,positive,because the rifle,0.2364


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [images|audio|videos|texts|keywords]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>SEARCH KEYWORD [images|audio|videos|texts|keywords]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __4. In Conclusion__

In this tutorial, we performed text vectorization using `movie review data`, and similar text search and keyword extraction. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adjusting various options, such as increasing the epoch or dataset size. Create your own model and provide competitive services by combining various unstructured data (image, audio, video, etc.) and structured data with ThanoSQL.
<br>
For the next step, explore the various "OPTIONS" and training methods of text vectorization models. If you want to learn more about building your own text model, proceed with the following tutorials.

* [How to Upload to ThanoSQL DB](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [Creating an Intermediate Similar Text Search Model]

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an text similarity search model: contact@smartmind.team</p>
</div>